In [ ]:
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
from torchvision import transforms

from PIL import Image
import matplotlib.pyplot as plt

from skimage.filters import threshold_otsu
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Load CNN and UNET
from CNN_Mask_to_Tens import CNN
from UNetwork import UNet

model = UNet()

In [ ]:
# Load custom data same way as during training is done, but now remember the paths as well

# Define transforms
transform = transforms.Compose([
    transforms.ToTensor(),
])
transform_mask = transforms.Compose([
    transforms.ToTensor(),
])

# In this example, the CustomDataset takes in two lists of file paths: image_list and mask_list, representing the paths to the images and their corresponding masks, 
# respectively. It also takes in an optional transform argument to apply any necessary transformations to the images and masks.
class CustomDataset(Dataset):
    def __init__(self, image_list, mask_list, transform=None, transform_mask=None):
        self.image_list = [os.path.join(image_list, f) for f in os.listdir(image_list)]
        self.mask_list = [os.path.join(mask_list, f) for f in os.listdir(mask_list)]
        self.transform = transform
        self.transform_mask = transform_mask

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_path = self.image_list[idx]
        mask_path = self.mask_list[idx]
        image = Image.open(image_path)
        image = image.convert('RGB')
        mask = Image.open(mask_path)
        mask = mask.convert('L')

        if self.transform:
            image = self.transform(image)
        if self.transform_mask:
            mask = self.transform_mask(mask)


        return image, mask, image_path, mask_path

folder_real = 'Realistic_Data/102_Real/'
folder_mask = 'Realistic_Data/102_Mask/'

data = CustomDataset(folder_real,folder_mask, transform=transform, transform_mask=transform_mask)

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)

# Define the data loader
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

len_train = len(train_loader)*batch_size
len_val = len(val_loader)*batch_size

# Mean and STD calculations for real images
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# Data normalization only based on training set
for i, (images, labels, image_paths, mask_paths) in enumerate(train_loader):
    psum    += images.sum(axis        = [0, 2, 3])
    psum_sq += (images ** 2).sum(axis = [0, 2, 3])
    

# Final Calculation
# Resolution of the image
image_size = 256
count = len_train * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

# Update the trasforms based on the computed mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=total_mean,
                            std=total_std)
])

transform_mask = transforms.Compose([
    transforms.ToTensor(),
])

# Reload data, now with normalization based on the training data
data = CustomDataset(folder_real,folder_mask, transform=transform, transform_mask=transform_mask)

# By using random seed 42 for both data splitting, it will result in the same split everytime.
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [ ]:
for i in range(0,len(test_data)):
    img, label, img_name, lab_name = test_data[i]
    ouput_folder = 'Generated_Masks/Model_102/'
    model = UNet()

    # Load the trained weights
    model.load_state_dict(torch.load('Unet_Simple_withtest/Realistic_102.pt'))
    model.eval()

    # Make a prediction
    with torch.no_grad():
        output = model(img.unsqueeze(0))

    # Convert the output tensor to a numpy array
    output_np = output.squeeze(0).squeeze(0).cpu().detach().numpy()

    # Apply a threshold to convert the output to a binary mask
    threshold_value = threshold_otsu(output_np)
    mask = (output_np >= threshold_value).astype(np.float32)

    # Save the output image
    output_filename = os.path.splitext(os.path.basename(img_name))[0] + "_output.jpg"
    output_path = os.path.join(ouput_folder, output_filename)
    
    # Save the image using plt.savefig()
    plt.imsave(output_path, mask)
    ouput_foldermask = 'Generated_Masks/Modelmask_102/'    
    output_filenamemask = os.path.splitext(os.path.basename(img_name))[0] + "_truemask.jpg"
    output_pathmask = os.path.join(ouput_foldermask, output_filenamemask)
    # Also save mask for comparison later on
    plt.imsave(output_pathmask, label.squeeze().numpy())

In [ ]:
transform_mask = transforms.Compose([
    transforms.ToTensor(),
])

masks = []
mask_show = []
for i in range(0,len(test_data)):
    img, label, img_name, lab_name = test_data[i]
    mask = Image.open(lab_name)
    mask = mask.convert('L')
    mask_show.append(mask)
    mask = transform_mask(mask)
    masks.append(mask)

# Print the number of loaded images
print("Number of loaded masks:", len(masks))


In [ ]:
def compute_pixel_accuracy(computed_mask, true_mask):
    """
    Computes the pixel accuracy based on the computed mask and true mask, considering
    only the values and indices where the true mask equals one.

    Args:
        computed_mask (np.array): Computed mask array.
        true_mask (np.array): True mask array.

    Returns:
        float: Pixel accuracy.
    """
    # Ensure the shapes of both masks are the same
    assert computed_mask.shape == true_mask.shape, "Mask shapes do not match."

    # Create a mask to filter indices where true mask equals one
    filter_mask = (true_mask == 1)

    # Filter computed mask and true mask using the filter mask
    computed_mask_filtered = computed_mask[filter_mask]
    true_mask_filtered = true_mask[filter_mask]

    # Calculate the number of correctly classified pixels
    num_correct_pixels = np.sum(computed_mask_filtered == true_mask_filtered)

    # Calculate the total number of pixels
    total_pixels = np.sum(filter_mask)

    # Compute pixel accuracy
    pixel_accuracy = num_correct_pixels / total_pixels

    return pixel_accuracy

In [ ]:
# This creates the binary mask as a numpy array
limit = 20
mask_bins =[]
comp_mask_bins = []

folder_pathmask = 'Generated_Masks\Modelmask_102'
for filename in os.listdir(folder_pathmask): 
    mask_path = os.path.join(folder_pathmask, filename)
    mask = Image.open(mask_path)
    mask_l = mask.convert('L')
    arrm_l = np.array(mask_l)
    threshold_value = threshold_otsu(arrm_l)
    mask_bin = (arrm_l >= threshold_value).astype(np.uint8)
    mask_bins.append(mask_bin)

# Create numpy array from computed mask by the model
# Best to have masks as output, or binarize them again
data =[]
avg = []
folder_path = 'Generated_Masks\Model_102'
i=0
for filename in os.listdir(folder_path):        
    image_path = os.path.join(folder_path, filename)
    image = Image.open(image_path)
    image_l = image.convert('L')
    arr_l = np.array(image_l)
    threshold_value = threshold_otsu(arr_l)
    comp_mask_bin = (arr_l >= threshold_value).astype(np.uint8)
    comp_mask_bins.append(comp_mask_bin)
    real_value = float(filename.split("_s")[1].split("_")[0])
    accuracy = compute_pixel_accuracy(comp_mask_bins[i], mask_bins[i])
    data.append((real_value, accuracy))
    i+=1

    # Create a dataframe from the data list
    df = pd.DataFrame(data, columns=["Sigma", "Accuracy Value"])

    # Print the modified DataFrame
print(df)

In [ ]:
plt.scatter(df['Sigma'], df['Accuracy Value'], marker='x')
plt.ylabel('Percentage [%]', fontsize=14)
plt.xlabel('Surface tension [mN/m]', fontsize=12)
plt.title('Accuracy model 510', fontsize=12)


In [ ]:
import numpy as np

sigmas = np.arange(30, 81, 1)  # Generate an array of sigma values from 30 to 80 with steps of 1
df['Sigma'] = pd.cut(df['Sigma'], bins=sigmas, right=False, labels=sigmas[:-1])  # Bin the Sigma values

average_accuracy = df.groupby("Sigma")["Accuracy Value"].mean().reset_index()


plt.scatter(df['Sigma'], df['Accuracy Value'], marker='x')
plt.plot(average_accuracy['Sigma'], average_accuracy['Accuracy Value'], 'C1')
plt.ylabel('Percentage [%]', fontsize=14)
plt.xlabel('Surface tension [mN/m]', fontsize=12)
plt.title('Average accuracy model 510', fontsize=12)


In [ ]:
average_accuracy = df.groupby("Sigma")["Accuracy Value"].mean().reset_index()
avg_total = df["Accuracy Value"].mean()
df_avg = pd.DataFrame(average_accuracy)
avg_total

In [ ]:
df.to_csv('PA_values/PA_102.csv', index=True)
df_avg.to_csv('PA_values/PA_avg_102.csv', index=True)

In [ ]:
df102 = pd.read_csv('PA_values/PA_102.csv')
df510 = pd.read_csv('PA_values/PA_510.csv')
df1020 = pd.read_csv('PA_values/PA_1020.csv')
df2040 = pd.read_csv('PA_values/PA_2040.csv')

# plt.scatter(df102['Sigma'], df102['Accuracy Value'], label='102', color = 'C3')
# plt.scatter(df510['Sigma'], df510['Accuracy Value'], label='510', color = 'C2')
plt.scatter(df1020['Sigma'], df1020['Accuracy Value'], label='1020', color = 'C1')
plt.scatter(df2040['Sigma'], df2040['Accuracy Value'], label='2040', color = 'C0')
plt.legend()
plt.title('Accuracies of Generated Mask')
plt.xlabel('Surface Tension [mN/m]')
plt.ylabel('Accuracy')
plt.savefig('PA_values/Best_Two')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
# Load random sample from test_data to check mask visually
img, label, img_name, lab_name = test_data[random.randint(0, len(test_data)-1)]
img = img.unsqueeze(0)

# Assuming you have the trained model and a test image

# Set the model to evaluation mode
model.eval()

# Perform forward pass on the test image
with torch.no_grad():
    output = model(img)  # Assuming you have the test image as a tensor
# Convert the output tensor to a numpy array and squeeze the batch and channel dimensions
output_np = output.squeeze(0).squeeze(0).cpu().numpy()
# Apply a threshold to convert the output to a binary mask
threshold_value =  threshold_otsu(output_np)
mask = (output_np >= threshold_value).astype(np.float32)


total_mean = np.array([0.4406, 0.4444, 0.3446])
total_std = np.array([0.2787, 0.2783, 0.3058])
# Invert the origial transform of the loaded tesor
invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = 1/total_std),
                                transforms.Normalize(mean = -total_mean,
                                                     std = [ 1., 1., 1. ]),
                               ])
inv_tensor = invTrans(img)

# Compute right order of tensor for plots
img = inv_tensor.permute(0, 2, 3, 1)
label = label.permute(1, 2, 0)

# Reshape the tensor
img = img.reshape(256, 256, 3)
label = label.reshape(256, 256, 1)

# Plot the original test image
plt.subplot(1, 3, 1)
plt.imshow(img)
plt.title('Original Image')
plt.axis('off')

# Plot the original mask
plt.subplot(1, 3, 2)
plt.imshow(label)
plt.title('Original Mask')
plt.axis('off')

# Plot the generated mask
plt.subplot(1, 3, 3)
plt.imshow(mask)
plt.title('Generated Mask')
plt.axis('off')
plt.show()